In [10]:
import psycopg2

# Connect to the database
conn = psycopg2.connect("dbname=hn user=julien")

# Create a cursor
cur = conn.cursor()

# Execute the query
cur.execute("""
    SELECT
        text, id, title
    FROM hn_article
    WHERE text IS NOT NULL
    ;
""")


# Fetch the results
resultsDB = cur.fetchall()



In [2]:
results = [result[0] for result in resultsDB]
ids = [result[1] for result in resultsDB]
""" results = results[:10000] """
len(results)



119151

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Create the vectorizer method
vectorizer = TfidfVectorizer(input="content", stop_words="english")

# It fits the text and transforms it
tfidf = vectorizer.fit_transform(results)

# Retrieve the terms found in the corpora
terms = vectorizer.get_feature_names_out()
print("Length of terms: ", len(terms))


Length of terms:  1973780


In [4]:
def get_keywords(id):
    index = ids.index(id)
    words = tfidf[index].toarray().argsort()[0][-10:]
    return terms[words]


def insert_tag_in_DB(id):
    tags = ",".join(get_keywords(id))
    try:
        cur.execute("""
            UPDATE hn_article
            SET tags = %s
            WHERE id = %s
            ;
        """, (tags, id))
        conn.commit()
    except Exception as e:
        print(e)
        conn.rollback()

In [11]:
for i in range(len(ids)):
    print("\r", i, "/", len(ids), end="")
    insert_tag_in_DB(ids[i])

 119150 / 119151